In [2]:
import pandas as pd

# Read the CSV data
df = pd.read_csv('../data/tv_export_data.csv')

# Ensure that 'detail_gender' is string type
df['detail_gender'] = df['detail_gender'].astype(str)

# # One-hot encode the 'detail_gender' column
# gender_df = pd.get_dummies(df['detail_gender'])

# # Display the DataFrame
# print(gender_df)


In [5]:
# Aggregate total episode count for each race and gender
episode_count_by_race_gender = df.groupby(['profile_race', 'detail_gender'])['episode_count'].sum()

# Display the result
print(episode_count_by_race_gender)


profile_race     detail_gender
asian            Female             69318
                 Male              108613
                 Undetermined         602
black            Female             72562
                 Male              184487
                 Undetermined        1768
indian           Female              6153
                 Male               13639
                 Undetermined           7
latino hispanic  Female             94031
                 Male              252543
                 Undetermined         165
middle eastern   Female              7003
                 Male              166623
                 Undetermined          25
white            Female            926829
                 Male             1608560
                 Undetermined        1533
Name: episode_count, dtype: int64


In [6]:
# Convert to DataFrame with one column per gender
episode_count_by_race_gender_df = episode_count_by_race_gender.unstack('detail_gender')

# Display the result
print(episode_count_by_race_gender_df)


detail_gender    Female     Male  Undetermined
profile_race                                  
asian             69318   108613           602
black             72562   184487          1768
indian             6153    13639             7
latino hispanic   94031   252543           165
middle eastern     7003   166623            25
white            926829  1608560          1533


In [7]:
import statsmodels.api as sm

# Define dependent variable
Y = episode_count_by_race_gender_df['Male']

# Define independent variables
X = episode_count_by_race_gender_df['Female']

# Add a constant to the independent variable
X = sm.add_constant(X)

# Conduct the ordinary least squares regression
model = sm.OLS(Y, X)
results = model.fit()

# Print out the statistics
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                   Male   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     400.3
Date:                Sat, 29 Jul 2023   Prob (F-statistic):           3.68e-05
Time:                        19:44:56   Log-Likelihood:                -73.975
No. Observations:                   6   AIC:                             151.9
Df Residuals:                       4   BIC:                             151.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6.244e+04   3.19e+04      1.959      0.1

c:\Users\cjong\Projects\python\scrotten_movies\.venv\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 6 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "


In [15]:
episode_count_by_race_gender_df_inverse = episode_count_by_race_gender_df.transpose()


In [16]:
episode_count_by_race_gender_df_inverse

profile_race,asian,black,indian,latino hispanic,middle eastern,white
detail_gender,,,,,,
Female,69318,72562,6153,94031,7003,926829
Male,108613,184487,13639,252543,166623,1608560
Undetermined,602,1768,7,165,25,1533


In [18]:
import statsmodels.api as sm

# Define dependent variable
Y = episode_count_by_race_gender_df_inverse['white']

# Define independent variables
X = episode_count_by_race_gender_df_inverse[['asian', 'black', 'indian', 'latino hispanic', 'middle eastern']]

# Add a constant to the independent variable
X = sm.add_constant(X)

# Conduct the ordinary least squares regression
model = sm.OLS(Y, X)
results = model.fit()

# Print out the statistics
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:                  white   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Sat, 29 Jul 2023   Prob (F-statistic):                nan
Time:                        20:17:17   Log-Likelihood:                 59.440
No. Observations:                   3   AIC:                            -112.9
Df Residuals:                       0   BIC:                            -115.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const              -0.0045        inf     

c:\Users\cjong\Projects\python\scrotten_movies\.venv\Lib\site-packages\statsmodels\stats\stattools.py:74: ValueWarning: omni_normtest is not valid with less than 8 observations; 3 samples were given.
  warn("omni_normtest is not valid with less than 8 observations; %i "
c:\Users\cjong\Projects\python\scrotten_movies\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:1794: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\Users\cjong\Projects\python\scrotten_movies\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:1794: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
c:\Users\cjong\Projects\python\scrotten_movies\.venv\Lib\site-packages\statsmodels\regression\linear_model.py:1716: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


In [21]:
import pandas as pd
import statsmodels.api as sm

# Assuming your data is saved in a CSV file named "tv_data.csv"
df = pd.read_csv("../data/tv_export_data.csv")

# Convert race into dummy variables
race_dummies = pd.get_dummies(df['profile_race'], prefix='race')
df = pd.concat([df, race_dummies], axis=1)

# Convert gender into a binary variable
df['profile_gender'] = df['profile_gender'].apply(lambda x: 1 if x == 'Female' else 0)

# Split dataframe by gender
df_male = df[df['profile_gender'] == 0]
df_female = df[df['profile_gender'] == 1]

# Run OLS regression on total episode count for each race
X_male = df_male[race_dummies.columns]
y_male = df_male['episode_count']
X_male = sm.add_constant(X_male)
model = sm.OLS(y_male, X_male)
results_male = model.fit()
print("OLS Regression Results for Males:")
print(results_male.summary())

# Run OLS regression on total episode count for each race
X_female = df_female[race_dummies.columns]
y_female = df_female['episode_count']
X_female = sm.add_constant(X_female)
model = sm.OLS(y_female, X_female)
results_female = model.fit()
print("\nOLS Regression Results for Females:")
print(results_female.summary())


ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).

In [22]:
import pandas as pd
import statsmodels.api as sm

# Assuming your DataFrame 'df' is already defined and has the required columns

races = ['white', 'black', 'asian', 'latino_hispanic', 'middle_eastern', 'indian']  # adjust this list based on the races represented in your data

for race in races:
    X = sm.add_constant(df['male_' + race])
    y = df['female_' + race]
    model = sm.OLS(y, X)
    results = model.fit()
    print(f"OLS Regression Results for {race.capitalize()} race:")
    print(results.summary())
    print("\n")


KeyError: 'male_white'